# More on training
This notebooks covers more detials on tweaking and optimizing the training process

## Optimizing the pipline
### Caching

In [ ]:
datasets = load_QM9_dataset('dsgn16/*', split_ratio={'train':8, 'test':2})
train = datasets['train']().repeat().batch(100)
output = pinn_network(train, **params)

### Preprocessing
You might see a notable difference in the preformance with and without preprocessing,  
especially when you are training with GPUs

**Note:** At this moment, you can not train forces with pre-precoessed distances

In [ ]:
datasets = load_QM9_dataset('dsgn16/*', split_ratio={'train':8, 'test':2})
train = datasets['train']().cache().repeat().batch(100)
output = pinn_network(train, **params)

In [ ]:
pre_fn = lambda t: pinn_network(t, pre_level=-10)
train = datasets['train']().repeat().batch(100).map(pre_fn)
output = pinn_network(train, pre_level=10, **params)

## Parallization with tf.Estimator

The estimator api makes it extremely easy to train on multiple GPUs.

In [ ]:
distribution = tf.contrib.distribute.MirroredStrategy(num_gpus=2)
config = tf.estimator.RunConfig(train_distribute=distribution)
# suppose you have two cards
params = {'network': {'func': 'pinn_network'}}
params['config'] = config
# Add the configuration to parameters
model = potential_model(params)